<a href="https://colab.research.google.com/github/Seiris21/2022_snowpack_capstone/blob/main/notebooks/data_ingestion/SnowCast%20Showdown%20Data%20Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!sudo apt-get update -y
!sudo apt-get upgrade -y
!pip3 install numpy==1.21
# Install GDAL and Geopandas
!sudo apt-get install libgdal-dev -y
!sudo apt install gdal-bin python-gdal python3-gdal --quiet -y
!sudo apt install python3-rtree --quiet -y
!pip3 install git+git://github.com/geopandas/geopandas.git --quiet

!pip3 install -U tornado

In [2]:
!python3 --version

Python 3.8.0


In [3]:
%%capture
%pip install "dask[complete]"
%pip install "dask[complete]" --upgrade

In [4]:
%%capture
%pip install pystac_client planetary_computer rasterio xarray-spatial

In [5]:
%%capture
!pip3 install matplotlib datetime pystac_client planetary_computer xarray datashader xarray-spatial 

In [ ]:
%%capture
!pip3 install rasterio geotiff geopy shapely imagecodecs

In [ ]:
%%capture
!pip3 install sklearn

In [ ]:
%%capture
!pip3 install requests earthengine-api datetime
!pip3 install earthengine-api --upgrade
!pip3 install httplib2==0.15.0

In [ ]:
%%capture
!pip3 install rioxarray

In [25]:
import os
from os import listdir
from os.path import isfile, join

import imagecodecs
from geotiff import GeoTiff
import geopy
import geopy.distance as distance
from shapely.geometry import Polygon

import re
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

from pystac_client import Client
import planetary_computer
import xarray
import dask.dataframe as dd
import xrspatial
from datashader.transfer_functions import shade, stack
from datashader.colors import Elevation
from datashader.utils import export_image

from sklearn.neighbors import BallTree

import shutil
import requests
import ee

import time 
import signal

import rioxarray
import rasterio
import rasterio.features
import shapely
from shapely import wkt


class TimeoutException(Exception):   # Custom exception class
    pass

def timeout_handler(signum, frame):   # Custom signal handler
    raise TimeoutException

# Data Ingestion and Exploration

In [26]:
trainfeatures = pd.read_csv("~/SnowData/Competition_Data/ground_measures_train_features.csv")

In [27]:
trainfeatures.head()

,Unnamed: 0,2013-01-01,2013-01-08,2013-01-15,2013-01-22,2013-01-29,2013-02-05,2013-02-12,2013-02-19,2013-02-26,...,2019-05-28,2019-06-04,2019-06-11,2019-06-18,2019-06-25,2019-12-03,2019-12-10,2019-12-17,2019-12-24,2019-12-31
0,CDEC:ADM,5.90,5.90,6.50,6.50,7.40,7.60,7.40,8.00,8.00,...,NaN,NaN,NaN,NaN,NaN,0.70,1.20,3.40,3.70,3.40
1,CDEC:AGP,17.52,17.54,17.85,17.39,18.03,17.70,17.65,16.66,17.21,...,NaN,NaN,NaN,NaN,NaN,0.00,0.60,0.20,NaN,NaN
2,CDEC:ALP,12.75,13.32,14.26,14.02,13.39,13.25,14.30,13.95,15.73,...,29.52,20.81,8.71,0.30,0.00,5.69,8.04,10.74,12.67,12.57
3,CDEC:BCB,4.30,4.42,4.62,4.53,4.67,4.90,4.90,5.06,5.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CDEC:BCH,2.88,3.00,3.48,3.84,3.96,4.44,5.40,5.16,3.60,...,0.84,0.60,0.36,0.36,0.24,2.88,4.56,4.68,5.04,6.00


In [28]:
trainfeatures = trainfeatures.melt(id_vars=['Unnamed: 0']).dropna().reset_index(drop = True)
trainfeatures.rename(columns = {'Unnamed: 0':"station_id", "variable":"date", "value":"SWE"}, inplace = True)
trainfeatures.head()

,station_id,date,SWE
0,CDEC:ADM,2013-01-01,5.90
1,CDEC:AGP,2013-01-01,17.52
2,CDEC:ALP,2013-01-01,12.75
3,CDEC:BCB,2013-01-01,4.30
4,CDEC:BCH,2013-01-01,2.88


In [29]:
#These are dates where no stations had information (Discovered after using KNN approach)
#Alternative method: Get all dates of cell_id samples, and compare those dates against what each station has for the interpolation
nan_dates = ['2013-04-03', '2013-04-29', '2013-05-03', '2013-05-25', '2013-06-01', '2013-06-08', '2016-02-08', '2016-03-26', '2016-04-01', '2016-04-03', '2016-04-04',
 '2016-04-07', '2016-04-16', '2016-05-09', '2016-05-27', '2016-06-26', '2017-01-28', '2017-01-29', '2018-03-04', '2018-03-30', '2018-03-31', '2018-04-22', '2018-04-23', 
 '2018-04-25', '2018-04-26', '2018-05-24', '2018-05-28', '2018-06-01', '2018-06-02', '2019-03-09', '2019-03-15', '2019-03-16', '2019-03-17', '2019-03-24', '2019-03-25', 
 '2019-03-29', '2019-04-07', '2019-04-08', '2019-04-17', '2019-04-18', '2019-04-19', '2019-04-21', '2019-04-27', '2019-04-28', '2019-05-01', '2019-05-02', '2019-05-03', 
 '2019-06-05', '2019-06-08', '2019-06-09', '2019-06-10', '2019-06-13', '2019-06-14', '2019-06-24', '2017-07-17', '2019-07-05', '2017-08-16', '2019-07-04', '2019-07-14',
 '2019-07-16', '2017-07-27', '2017-07-19', '2019-07-03', '2017-08-15', '2019-07-13', '2019-07-15', '2017-07-18', '2016-07-08']

In [126]:
#Linear regression implementation, filling in based on nan_dates list
supplement = []

#Iterate through all the unique stations
for station in trainfeatures['station_id'].unique(): #['CDEC:SSM']: #
  #Get subset for this station
  subset = trainfeatures[trainfeatures['station_id']==station].copy()
  #make filler rows with missing dates
  filler = [[station,date,np.nan] for date in nan_dates]
  #Append filler rows to subset and sort on date and reset index
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
  #print(station,len(subset.index))
  #print(subset.head())
  for date in nan_dates:
    #Find NaN date
    nan_index = subset.index[subset['date'] == date].tolist()[0]
    nan_date = datetime.strptime(date,'%Y-%m-%d')
    
    #There is a conditional needed for stations that stopped reporting before 2019
    try:
      count=0
      #Find older date that HAS value. Sometimes needed because filler inserted NaNs
      while subset.iloc[nan_index-1-count].isnull().any():
        count+=1
      #Older date (nan-1)
      if (nan_index-1-count)>=0:
        older_date = datetime.strptime(subset.iloc[nan_index-1-count]['date'],'%Y-%m-%d')
        older_swe = subset.iloc[nan_index-1-count]['SWE']
      else:
        older_date = datetime.strptime(subset.iloc[nan_index-1]['date'],'%Y-%m-%d')
        older_swe = np.nan
      #print('Older',nan_index-1,older_date,older_swe)
      #print('NaN-inserted',nan_index,nan_date)

      #Newer date is next date that HAS value, otherwise enter except
      counter=0
      while subset.iloc[nan_index+1+counter].isnull().any():
        counter+=1
      #Newer date
      newer_date = datetime.strptime(subset.iloc[nan_index+1+counter]['date'],'%Y-%m-%d')
      newer_swe = subset.iloc[nan_index+1+counter]['SWE']
      #print('newer',nan_index+1+counter,newer_date,newer_swe)
      #print('______________________________')

      #Change per day
      delta_day = (newer_swe-older_swe)/(newer_date-older_date).days

      #Add expected change to older swe
      est_swe = older_swe + (delta_day*(nan_date-older_date).days)

      #Add "entry" row to supplement
      supplement.append([station,date,est_swe])
    #IndexError happens when the last date is actually from the nan list. Because of this, We DEFINITELY need to do some inter-station interpolation
    except IndexError:
      supplement.append([station,date,np.nan])

#Problem with simple linear interpolation: There are large enough gaps that the "missing days" in the data sometimes are the closest dates to themselves

/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and wi

/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and wi

/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and wi

/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and wi

/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and wi

...

/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and wi

/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and wi

/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and wi

/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and wi

/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
/tmp/ipykernel_6501/1846747857.py:11: FutureWarning: The frame.append method is deprecated and wi

In [127]:
#Add incomplete supplement (testing to see how many knn nans are filled in)
trainfeatures = trainfeatures.append(pd.DataFrame(supplement, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
trainfeatures.head()

/tmp/ipykernel_6501/4277137967.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trainfeatures = trainfeatures.append(pd.DataFrame(supplement, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)


,station_id,date,SWE
0,CDEC:ADM,2013-01-01,5.9
1,SNOTEL:628_UT_SNTL,2013-01-01,9.8
2,SNOTEL:629_CO_SNTL,2013-01-01,3.6
3,SNOTEL:633_CA_SNTL,2013-01-01,9.4
4,SNOTEL:637_ID_SNTL,2013-01-01,6.9


In [31]:
trainmeta = pd.read_csv("~/SnowData/Competition_Data/ground_measures_metadata.csv")
trainmeta.head()

,station_id,name,elevation_m,latitude,longitude,state
0,CDEC:ADM,Adin Mountain,1889.76,41.237000,-120.792000,California
1,CDEC:AGP,Agnew Pass,2880.36,37.726631,-119.141731,California
2,CDEC:ALP,Alpha (Smud),2316.48,38.804192,-120.215652,California
3,CDEC:BCB,Blackcap Basin,3139.44,37.066685,-118.773010,California
4,CDEC:BCH,Beach Meadows,2331.72,36.126095,-118.293457,California


In [32]:
trainfeatures = trainfeatures.merge(trainmeta, how = 'left', on='station_id')
trainfeatures.head()

,station_id,date,SWE,name,elevation_m,latitude,longitude,state
0,CDEC:ADM,2013-01-01,5.90,Adin Mountain,1889.76,41.237000,-120.792000,California
1,CDEC:AGP,2013-01-01,17.52,Agnew Pass,2880.36,37.726631,-119.141731,California
2,CDEC:ALP,2013-01-01,12.75,Alpha (Smud),2316.48,38.804192,-120.215652,California
3,CDEC:BCB,2013-01-01,4.30,Blackcap Basin,3139.44,37.066685,-118.773010,California
4,CDEC:BCH,2013-01-01,2.88,Beach Meadows,2331.72,36.126095,-118.293457,California


In [33]:
gridcells = gpd.read_file('~/SnowData/Competition_Data/grid_cells.geojson')
print(gridcells.head())

                                cell_id           region  \
0  0003f387-71c4-48f6-b2b0-d853bd4f0aba          sierras   
1  000617d8-8c14-43e2-b708-7e3a69fe3cc3  central rockies   
2  000863e7-21e6-477d-b799-f5675c348627            other   
3  000ba8d9-d6d5-48da-84a2-1fa54951fae1          sierras   
4  00146204-d4e9-4cd8-8f86-d1ef133c5b6d          sierras   

                                            geometry  
0  POLYGON ((-118.71895 37.07419, -118.71895 37.0...  
1  POLYGON ((-107.07679 37.78042, -107.07679 37.7...  
2  POLYGON ((-119.40167 37.02400, -119.40167 37.0...  
3  POLYGON ((-119.32082 37.43171, -119.32082 37.4...  
4  POLYGON ((-118.52132 36.65735, -118.52132 36.6...  


In [34]:
traindf = pd.read_csv("~/SnowData/Competition_Data/train_labels.csv")

traindf = traindf.melt(id_vars=["cell_id"]).dropna().reset_index(drop = True)
traindf.rename(columns = {"cell_id":"cell_id", "variable":"date", "value":"SWE"}, inplace = True)

traindf = traindf.merge(gridcells, how = 'left', on='cell_id')


traindf = gpd.GeoDataFrame(traindf, crs ="EPSG:4326")
traindf

,cell_id,date,SWE,region,geometry
0,00c4db22-a423-41a4-ada6-a8b1b04153a4,2013-01-01,12.7,other,"POLYGON ((-121.93492 41.16327, -121.93492 41.1..."
1,018cf1a1-f945-4097-9c47-0c4690538bb5,2013-01-01,20.4,sierras,"POLYGON ((-120.61440 39.67242, -120.61440 39.6..."
2,01be2cc7-ef77-4e4d-80ed-c4f8139162c3,2013-01-01,37.0,sierras,"POLYGON ((-119.60829 38.27575, -119.60829 38.2..."
3,02c3ec4a-8de4-4284-9ec1-5a942d3d098e,2013-01-01,2.3,other,"POLYGON ((-107.19357 44.57879, -107.19357 44.5..."
4,02cf33c2-c8e2-48b9-bf72-92506e97e251,2013-01-01,8.0,central rockies,"POLYGON ((-106.60068 40.39461, -106.60068 40.4..."
...,...,...,...,...,...
91485,fd4492f2-8aa9-4279-bdc0-73991786943f,2019-12-31,1.3,central rockies,"POLYGON ((-105.07354 38.87270, -105.07354 38.8..."
91486,fde3221a-9ce3-45a9-857f-bd196b07aa05,2019-12-31,5.6,central rockies,"POLYGON ((-106.10661 39.29804, -106.10661 39.3..."
91487,fdeb8912-f9d1-445d-aadb-e943534f67fe,2019-12-31,8.8,central rockies,"POLYGON ((-107.92120 37.79462, -107.92120 37.8..."
91488,fe33672e-7ea7-4c5d-8639-96b2cc7edb0c,2019-12-31,2.9,other,"POLYGON ((-122.02475 43.89659, -122.02475 43.9..."


In [3]:
print(len(traindf.loc[pd.to_datetime(traindf.date) > datetime.strptime("2016-01-01", "%Y-%m-%d"), "region"]))
print(len(traindf.loc[traindf["region"] == "sierras", "region"]))

NameError: name 'traindf' is not defined

In [35]:
print(len(traindf))
traindf = traindf.loc[pd.to_datetime(traindf.date) >= datetime.strptime("2016-01-01", "%Y-%m-%d")].reset_index(drop = True)
traindf = traindf.loc[traindf["region"] == "sierras"].reset_index(drop = True)
print(len(traindf["region"]))
traindf.head()

91490
32164


,cell_id,date,SWE,region,geometry
0,018cf1a1-f945-4097-9c47-0c4690538bb5,2016-01-05,16.4,sierras,"POLYGON ((-120.61440 39.67242, -120.61440 39.6..."
1,01be2cc7-ef77-4e4d-80ed-c4f8139162c3,2016-01-05,21.1,sierras,"POLYGON ((-119.60829 38.27575, -119.60829 38.2..."
2,04b7603c-26c6-4004-a8bc-58b2b60e810e,2016-01-05,3.5,sierras,"POLYGON ((-118.34166 36.56361, -118.34166 36.5..."
3,147d5eb4-e574-47e4-994a-8a2908c06050,2016-01-05,11.7,sierras,"POLYGON ((-120.87491 39.78297, -120.87491 39.7..."
4,174e3100-c30e-46a4-ac7c-30cd521fc390,2016-01-05,12.9,sierras,"POLYGON ((-119.54540 37.63117, -119.54540 37.6..."


In [36]:
gdf = gpd.GeoDataFrame(trainmeta, 
                       geometry = gpd.points_from_xy(trainmeta.longitude, trainmeta.latitude),
                       crs = "EPSG:4326")

gdf

,station_id,name,elevation_m,latitude,longitude,state,geometry
0,CDEC:ADM,Adin Mountain,1889.760000,41.237000,-120.792000,California,POINT (-120.79200 41.23700)
1,CDEC:AGP,Agnew Pass,2880.360000,37.726631,-119.141731,California,POINT (-119.14173 37.72663)
2,CDEC:ALP,Alpha (Smud),2316.480000,38.804192,-120.215652,California,POINT (-120.21565 38.80419)
3,CDEC:BCB,Blackcap Basin,3139.440000,37.066685,-118.773010,California,POINT (-118.77301 37.06668)
4,CDEC:BCH,Beach Meadows,2331.720000,36.126095,-118.293457,California,POINT (-118.29346 36.12609)
...,...,...,...,...,...,...,...
695,SNOTEL:989_ID_SNTL,Moscow Mountain,1432.560059,46.805000,-116.853500,Idaho,POINT (-116.85350 46.80500)
696,SNOTEL:990_WA_SNTL,Beaver Pass,1106.423950,48.879299,-121.255501,Washington,POINT (-121.25550 48.87930)
697,SNOTEL:992_UT_SNTL,Bear River RS,2675.229492,40.885201,-110.827698,Utah,POINT (-110.82770 40.88520)
698,SNOTEL:998_WA_SNTL,Easy Pass,1606.296021,48.859329,-121.438950,Washington,POINT (-121.43895 48.85933)


# Bringing in ASO Data

In [37]:
mypath = "/home/ubuntu/SnowData/ASO_Tifs"

files = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
files = [f for f in files if ".xml" not in f]

print(files[0:5])

['/home/ubuntu/SnowData/ASO_Tifs/ASO_50M_SWE_USCARC_20170717.tif', '/home/ubuntu/SnowData/ASO_Tifs/ASO_50M_SWE_USCAKC_20190428.tif', '/home/ubuntu/SnowData/ASO_Tifs/ASO_50M_SWE_USCAKW_20190324.tif', '/home/ubuntu/SnowData/ASO_Tifs/ASO_50M_SWE_USCATE_20190705.tif', '/home/ubuntu/SnowData/ASO_Tifs/ASO_50M_SWE_USCATB_20170816.tif']


In [41]:
train_add = []
x=0

for k in files:
    geo_tiff = GeoTiff(k)

    zarr_array = geo_tiff.read()

    array = np.array(zarr_array)

    width = 20 * (array.shape[0] // 20)
    height = 20 * (array.shape[1] // 20)
    
    y = 0


    for i in range(0, width, 20):
        for j in range(0, height, 20):
            #Assume each site is 100m (question not clear)
            d = distance.distance(kilometers=1)

            #get the date and cell_id from the string
            date = k[-12:-8] +'-'+k[-8:-6] +'-'+k[-6:-4]

            cell_id = k[-k.rfind('/')-1:-13] + '_' + str(y)

            # Going clockwise, from lower-left to upper-left, upper-right...
            coords = geo_tiff.get_wgs_84_coords(i,j)
            p1 = geopy.Point((coords[1], coords[0]))
            p2 = d.destination(point=p1, bearing=180)
            p3 = d.destination(point=p2, bearing=90)
            p4 = d.destination(point=p3, bearing=0)

            points = [(p.longitude, p.latitude) for p in [p1,p2,p3,p4]]
            polygon = Polygon(points)

            area_box = [(p1.longitude, p1.latitude), (p3.longitude, p3.latitude)]
            array = geo_tiff.read_box(area_box)
            array[array == -9999] = np.nan
            try:
                mean = np.nanmean(array)*39.3701 #metric to inches conversion
            except Exception as e:
                mean = np.nan
            
            row= [cell_id, date, mean, polygon]
            train_add.append(row)
            y+=1
    x += 1
    print(f'{x} out of {len(files)} images complete')

train_add = gpd.GeoDataFrame(pd.DataFrame(data = train_add, columns = ['cell_id', 'date', 'SWE', 'geometry']))            

/tmp/ipykernel_15310/3788606662.py:41: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(array)*39.3701 #metric to inches conversion


1 out of 94 images complete
2 out of 94 images complete
3 out of 94 images complete
4 out of 94 images complete
5 out of 94 images complete
6 out of 94 images complete
7 out of 94 images complete
8 out of 94 images complete
9 out of 94 images complete
10 out of 94 images complete
11 out of 94 images complete
12 out of 94 images complete
13 out of 94 images complete
14 out of 94 images complete
15 out of 94 images complete
16 out of 94 images complete
17 out of 94 images complete
18 out of 94 images complete
19 out of 94 images complete
20 out of 94 images complete
21 out of 94 images complete
22 out of 94 images complete
23 out of 94 images complete
24 out of 94 images complete
25 out of 94 images complete
26 out of 94 images complete


/home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


27 out of 94 images complete
28 out of 94 images complete
29 out of 94 images complete
30 out of 94 images complete
31 out of 94 images complete
32 out of 94 images complete
33 out of 94 images complete
34 out of 94 images complete
35 out of 94 images complete
36 out of 94 images complete
37 out of 94 images complete
38 out of 94 images complete
39 out of 94 images complete
40 out of 94 images complete
41 out of 94 images complete
42 out of 94 images complete
43 out of 94 images complete
44 out of 94 images complete
45 out of 94 images complete
46 out of 94 images complete
47 out of 94 images complete
48 out of 94 images complete
49 out of 94 images complete
50 out of 94 images complete
51 out of 94 images complete
52 out of 94 images complete
53 out of 94 images complete
54 out of 94 images complete
55 out of 94 images complete
56 out of 94 images complete
57 out of 94 images complete
58 out of 94 images complete
59 out of 94 images complete
60 out of 94 images complete
61 out of 94 i

In [42]:
train_add.dropna(inplace = True)
train_add["region"] = "sierras"

train_add = train_add.loc[:,['cell_id','date','SWE','region','geometry']].reset_index(drop = True)

In [43]:
traindf = pd.concat([traindf, train_add]).reset_index(drop=True)

In [61]:
traindf = traindf.loc[pd.to_datetime(traindf.date) >= datetime.strptime("2016-01-01", "%Y-%m-%d")] \
    .loc[traindf["SWE"] >= 0] \
    .reset_index(drop = True) 
    
print(traindf.date.min())
print(len(traindf))

2016-01-05
129805


In [62]:
pd.DataFrame(traindf).to_csv('~/SnowData/traindf_allregions.csv')

# Adding Station Data

In this section we will take the measurements of ground stations and add those as features to our data



In [142]:
trainfeatures = trainfeatures[['station_id',	'date',	'SWE',	'name']]

### Balltree/KNN approach

In [143]:

#Adapted from AutoGIS| University of Helsinki
# https://automating-gis-processes.github.io/site/notebooks/L3/nearest-neighbor-faster.html
def get_knearest(src_points, candidates, knn=1):
  '''
  K nearest neighbors for every source point given candidate points
  '''
  #Make candidates BallTree format
  tree = BallTree(candidates,leaf_size=15,metric='haversine')

  #Find closest points
  distances, indices = tree.query(src_points, k=knn)

  #Transpose into arrays
  distances = distances.transpose()
  indices = indices.transpose()

  #neighbor_idx = []
  #neighbor_dist = []
   
  return(indices, distances)
  #Iterate for k neighbors
  #for i in range(knn):
  #  neighbor_idx.append(indices[i])
  #  neighbor_dist.append(distances[i])
  #Return list of lists in order of KNN
  #return(neighbor_idx,neighbor_dist)



  return distances,indices

def nearest_neighbor(left_gdf, right_gdf, return_dist=False, knn=1):
  """
  For each point in left_gdf, find closest point in right GeoDataFrame and return them.

  NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
  """
  #Some Nan buffer to KNN search
  knn = knn*3

  left_geom_col = left_gdf.geometry.name
  right_geom_col = right_gdf.geometry.name

  # Ensure that index in right gdf is formed of sequential numbers
  right = right_gdf.copy().reset_index(drop=True)

  # Parse coordinates from points and insert them into a numpy array as RADIANS
  # For left radians, data is in polygon format, so apply meter crs, get centroid, and revert
  left_radians = np.array(left_gdf[left_geom_col].to_crs('epsg:4087').centroid.to_crs("EPSG:4326").apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
  right_radians = np.array(right[right_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())

  # Find the nearest points
  # -----------------------
  # closest ==> index in right_gdf that corresponds to the closest point
  # dist ==> distance between the nearest neighbors (in meters)

  closest, dist = get_knearest(src_points=left_radians, candidates=right_radians, knn=knn)

  #return(closest,dist)
    
  closest_points = gpd.GeoDataFrame()
    
  #Loop for knn
  for i in range(knn):
    # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
    #Loop to return closest starting from 0 idx
    closest_points['station_id_'+str(i)] = right['station_id'].loc[closest[i]].values
    closest_points['elevation_m_'+str(i)] = right['elevation_m'].loc[closest[i]].values

    # Add distance if requested
    if return_dist:
      # Convert to meters from radians
      earth_radius = 6371000  # meters
      closest_points['distance_'+str(i)] = dist[i] * earth_radius

  return closest_points

def inverseDmean(df,power):
  #Formula for inverse Distance Average = ((x1/d1^p)+(x2/d2^p)....)/((1/d1^p)+(1/d2^p)....)
  #https://gisgeography.com/inverse-distance-weighting-idw-interpolation/
  subset = df.filter(regex='distance_[0-9]+|SWE_[0-9]+')
  numerator = pd.DataFrame()
  denominator = pd.DataFrame()

  for i in range(int(subset.shape[1]/2)):
    numerator['x_'+str(i)]=subset['SWE_'+str(i)]/(subset['distance_'+str(i)]**power)
    denominator['x_'+str(i)]=1/(subset['distance_'+str(i)]**power)

  #There are cells without SWE data. We do not want this in the inverse Distance Calculation
  nulls = np.where(pd.isnull(numerator))
  for row,column in zip(nulls[0],nulls[1]):
    denominator.at[row, denominator.columns[column]] = np.nan
  
  numerator['sum']=numerator.sum(axis=1)
  #print(numerator.head())
  denominator['sum']=denominator.sum(axis=1)
  #print(denominator.head())

  return(numerator['sum']/denominator['sum'])

def swe_calculation(train, labels, closest_stations, knn=1):
  #Join labels with closest_stations
  labels_joined = labels.join(closest_stations)

  #Prepare column names
  SWE_names=[]
  elevation_names=[]
  reordered_columns = ['cell_id', 'date', 'SWE', 'region', 'geometry',
                       'mean_inversed_swe', 'mean_local_swe',	'median_local_swe',	'max_local_swe', 'min_local_swe',
                       'mean_local_elevation',	'median_local_elevation',	'max_local_elevation','min_local_elevation']
  for i in range(knn):
    reordered_columns.extend(['station_id_'+str(i),'elevation_m_'+str(i),'distance_'+str(i),'SWE_'+str(i)])
    SWE_names.append('SWE_'+str(i))
    elevation_names.append('elevation_m_'+str(i))
  
  #Merge against cell_id+date to get closest stations for each cell
  idx = 0
  for i in range(knn*3):
    train
    if i == 0:
      tmp_merged = pd.merge(labels_joined, train, how="left", left_on=['station_id_'+str(i), 'date'], right_on=['station_id','date'],suffixes=(None,'_'+str(i))).drop(columns= ['station_id'])
    else:
      tmp_merged = pd.merge(tmp_merged, train, how="left", left_on=['station_id_'+str(i), 'date'], right_on=['station_id','date'],suffixes=(None,'_'+str(i))).drop(columns= ['station_id'])

  #Filter out nearest neighbors with NaN, get 5 closest WITH VALUES
  filtered = []
  for idx,row in tmp_merged.iterrows():
    index = []
    values = []
    i=0
    counter=0
    while i<knn:
      if not pd.isna(row['SWE_'+str(counter)]):
        i+=1
        index.append(counter)
      counter+=1
    for j in index:
      values.extend([row['station_id_'+str(j)], row['elevation_m_'+str(j)], row['distance_'+str(j)], row['SWE_'+str(j)]])
    filtered.append(values)

  #Re-merge with cell data
  merged_train = labels.join(pd.DataFrame(filtered,columns=reordered_columns[-4*knn:]))

  #Calculations
  #Elevations
  # Normal Mean
  merged_train['mean_local_elevation']=merged_train[elevation_names].mean(axis=1)
  # Median
  merged_train['median_local_elevation']=merged_train[elevation_names].median(axis=1)
  # Max
  merged_train['max_local_elevation']=merged_train[elevation_names].max(axis=1)
  # Min
  merged_train['min_local_elevation']=merged_train[elevation_names].min(axis=1)

  #SWE
  #Inverse Distance Mean
  merged_train['mean_inversed_swe']=inverseDmean(merged_train,2)
  #Normal Mean
  merged_train['mean_local_swe']=merged_train[SWE_names].mean(axis=1)
  #Median
  merged_train['median_local_swe']=merged_train[SWE_names].median(axis=1)
  #Min
  merged_train['min_local_swe']=merged_train[SWE_names].min(axis=1)
  #Max
  merged_train['max_local_swe']=merged_train[SWE_names].max(axis=1)

  #Reorder Columns
  merged_train=merged_train[reordered_columns]

  return(merged_train)

In [144]:
knn=5
#DO NOT want traindf.date or trainfeatures in datetime format

closest_stations = nearest_neighbor(traindf, gdf, return_dist=True,knn=knn)
traindf = swe_calculation(train=trainfeatures, labels=traindf, closest_stations=closest_stations, knn=knn)

In [145]:
traindf.sample(frac = .1).head()

,cell_id,date,SWE,region,geometry,mean_inversed_swe,mean_local_swe,median_local_swe,max_local_swe,min_local_swe,...,distance_2,SWE_2,station_id_3,elevation_m_3,distance_3,SWE_3,station_id_4,elevation_m_4,distance_4,SWE_4
71650,ASO_50M_SWE_USCAMB4060,2019-03-29,53.252170,sierras,"POLYGON ((-119.58710 37.87948, -119.58710 37.8...",55.227979,56.605143,50.512857,79.095714,35.811429,...,14973.324625,50.512857,CDEC:HRS,2560.3200,17951.307334,79.095714,CDEC:GIN,2148.840000,22117.152043,35.811429
73187,ASO_50M_SWE_USCATB0460,2018-05-28,0.117197,sierras,"POLYGON ((-119.39592 38.02995, -119.39592 38.0...",1.634396,2.290922,0.162857,11.001751,0.000000,...,16382.266404,11.001751,CDEC:VRG,2834.6400,17596.879698,0.200000,SNOTEL:587_CA_SNTL,2819.095215,22638.771013,0.000000
118022,ASO_50M_SWE_USCASF1560,2017-07-18,19.256929,sierras,"POLYGON ((-118.77410 37.12138, -118.77410 37.1...",5.738775,5.264542,4.840000,13.756154,0.733478,...,19216.631169,4.840000,CDEC:VLC,3063.2400,21019.081759,4.989167,CDEC:MTM,3017.520000,21259.450531,2.003913
33920,ASO_50M_SWE_USCAKC0100,2019-04-28,0.000000,sierras,"POLYGON ((-118.82430 37.17371, -118.82430 37.1...",38.190275,38.854857,39.894286,58.714286,11.254286,...,15164.147365,39.894286,CDEC:RCK,2956.5600,18008.479379,11.254286,CDEC:MTM,3017.520000,25991.599855,58.714286
4072,836b3ae7-09c5-46db-9f9e-5df8544aa734,2016-05-09,48.800000,sierras,"POLYGON ((-119.49150 38.13457, -119.49150 38.1...",6.718008,8.960000,13.642857,17.442857,0.000000,...,14862.753623,13.714286,CDEC:SDW,2838.6024,14876.935902,13.642857,SNOTEL:771_CA_SNTL,2673.095947,14953.205760,17.442857


In [63]:
traindf = traindf[['cell_id','date','SWE','region','geometry','mean_inversed_swe',
                   'mean_local_swe','median_local_swe','max_local_swe','min_local_swe',
                   'mean_local_elevation','median_local_elevation','max_local_elevation','min_local_elevation',]]
                   
print(traindf.isna().sum())
pd.DataFrame(traindf).to_csv('~/SnowData/traindf_allregions.csv')

cell_id                   0
date                      0
SWE                       0
region                    0
geometry                  0
mean_inversed_swe         0
mean_local_swe            0
median_local_swe          0
max_local_swe             0
min_local_swe             0
mean_local_elevation      0
median_local_elevation    0
max_local_elevation       0
min_local_elevation       0
dtype: int64


# Modis Data

## Environment Setup

In [66]:
traindf = pd.read_csv('~/SnowData/traindf_allregions.csv').drop('Unnamed: 0', axis = 1)
traindf['geometry'] = traindf['geometry'].apply(wkt.loads)

traindf = gpd.GeoDataFrame(traindf, geometry = 'geometry' , crs ="EPSG:4326")
traindf.head()
traindf.tail()

,cell_id,date,SWE,region,geometry,mean_inversed_swe,mean_local_swe,median_local_swe,max_local_swe,min_local_swe,mean_local_elevation,median_local_elevation,max_local_elevation,min_local_elevation,copernicus_filelocations
129800,ASO_50M_SWE_USCALB_79,2018-06-01,0.474991,sierras,"POLYGON ((-118.99236 37.58734, -118.99236 37.5...",2.126230,1.906877,1.888571,3.574384,0.0,2808.36624,2880.36,3063.24,2307.0312,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...
129801,ASO_50M_SWE_USCALB_87,2018-06-01,1.828016,sierras,"POLYGON ((-118.98152 37.60555, -118.98152 37.5...",2.115175,1.906877,1.888571,3.574384,0.0,2808.36624,2880.36,3063.24,2307.0312,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...
129802,ASO_50M_SWE_USCALB_88,2018-06-01,0.114890,sierras,"POLYGON ((-118.98128 37.59654, -118.98128 37.5...",2.125414,1.906877,1.888571,3.574384,0.0,2808.36624,2880.36,3063.24,2307.0312,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...
129803,ASO_50M_SWE_USCALB_89,2018-06-01,2.170014,sierras,"POLYGON ((-118.98104 37.58753, -118.98104 37.5...",2.135945,1.906877,1.888571,3.574384,0.0,2808.36624,2880.36,3063.24,2307.0312,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...
129804,ASO_50M_SWE_USCALB_99,2018-06-01,2.213607,sierras,"POLYGON ((-118.96972 37.58772, -118.96972 37.5...",2.176050,1.906877,1.888571,3.574384,0.0,2808.36624,2880.36,3063.24,2307.0312,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...


In [67]:
signal.signal(signal.SIGALRM, timeout_handler)

traindf["date"] = pd.to_datetime(traindf.date)

#I am creating a string version of the date to use as a filename
traindf["datestring"] = traindf.date.map(lambda d: str(d.year)+d.strftime('%j'))

#Now I calculate my centroid from the provided geometry
#Ignore the warnings this creates. It is in a projected crs
traindf["centroid"] = traindf.geometry.to_crs('+proj=cea').centroid
traindf["center_lat"] = traindf.centroid.y
traindf["center_long"] = traindf.centroid.x

#Logging in to Earth Engine
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

/tmp/ipykernel_15310/2857808034.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  traindf["center_lat"] = traindf.centroid.y
/tmp/ipykernel_15310/2857808034.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  traindf["center_long"] = traindf.centroid.x


In [ ]:
'''
def pull_MODIS(traindf, modis, overwrite = False, names_only = False):
  filelocations = []
  x = 0

  for i in range(len(traindf.SWE)):

    #create a name for the image
    pict_name = traindf.cell_id[i] + '_' + modis + '_' + traindf.datestring[i] + '.jpg'

    #create the whole filename with path to the correct folder
    filename = os.path.join('/content/', modis, pict_name)

    if names_only:
      filelocations.append(filename)
      x += 1
      if x % 5000 == 0:
        print(f'{x} files already exist')

    elif os.path.exists(filename) and not overwrite:
      filelocations.append(filename)
      x += 1
      if x % 5000 == 0:
        print(f'{x} files already exist')

    else:
      #We need a start date and an end date. Just like a regular python slice, 
      #the end date is not included, so by using a 1 day frame, I am actually limiting
      #the range to only the day in question
      start_date = traindf.date[i] - timedelta(days = 7)
      end_date = traindf.date[i] + timedelta(days = 1)

      #First I get the image collection from the MODIS data, filter it only to the days in question
      #and select my bands, then sort so the most recent day in the group is at the top
      Collection = ee.ImageCollection(f'MODIS/006/{modis}') \
                  .filter(ee.Filter.date(start_date, end_date)) \
                  .filter(ee.Filter.notNull(['system:index'])) \
                  .select(['NDSI_Snow_Cover', 'Snow_Albedo_Daily_Tile', 'NDSI']) \
                  .sort('system:index', False) 

      #I create a google earth images point based on the area centroid
      centroid = ee.Geometry.Point(traindf.center_long[i], traindf.center_lat[i])

      #Because the image collection is limited to a single day, there is only one image
      #So I just take it
      point = Collection.first().unmask(0)

      # Get individual band arrays and build them into an RGB image
      # The "buffer" is a circular distance around the point, measured in meters right now it is 100km
      rgb = ee.Image.rgb(point.clip(centroid.buffer(10000)).select('NDSI_Snow_Cover').divide(100), #I divide by 100 to get it between 0 and 1
                        point.clip(centroid.buffer(10000)).select('Snow_Albedo_Daily_Tile').divide(100), #I divide by 100 to get it between 0 and 1
                        point.clip(centroid.buffer(10000)).select('NDSI').divide(10000)).visualize() #I divide by 10000 to get it between 0 and 1

      #Now I get the url for the image
      url = rgb.getThumbURL()

      #add the name to my list I created earlier
      filelocations.append(filename)

      #now I open the url and download the image to the specified file location
      response = requests.get(url, stream=True)
      with open(filename, 'wb') as out_file:
          shutil.copyfileobj(response.raw, out_file)
      del response
    
  #traindf[f"{modis}_filelocations"] = filelocations
'''

In [68]:
def pull_MODIS_list(traindf, modis, signal_timer = 5):
  datalist = []
  x= 0

  still_working = True
  while still_working:
    try:
      Collection = ee.ImageCollection(f'MODIS/006/{modis}') \
                  .select(['NDSI_Snow_Cover', 'Snow_Albedo_Daily_Tile', 'NDSI'])
      
    except Exception as e:
      print("Some Error with Image Collection")
    else: 
      still_working = False        
  

  for i in range(len(traindf.SWE)):
    still_working = True
    while still_working:
      signal.alarm(signal_timer)
      try:
        row = [traindf.cell_id[i], traindf.date[i]]

        #We need a start date and an end date. Just like a regular python slice, 
        #the end date is not included, so by using a 1 day frame, I am actually limiting
        #the range to only the day in question
        start_date = traindf.date[i] - timedelta(days = 7)
        end_date = traindf.date[i] + timedelta(days = 1)

        #First I get the image collection from the MODIS data, filter it only to the days in question
        #and select my bands, then sort so the most recent day in the group is at the top
        DatedCollection = Collection.filter(ee.Filter.date(start_date, end_date)) \
                                    .filter(ee.Filter.notNull(['system:index'])) \
                                    .sort('system:index', False)

        #Because the image collection is limited to a single day, there is only one image
        #So I just take it
        point = DatedCollection.first().unmask(0)

        aoi = ee.Geometry.Polygon(list(traindf.iloc[i].geometry.exterior.coords))

        bands = point.reduceRegion(reducer = ee.Reducer.mean(),
        geometry= aoi)

        bands = bands.toArray(['NDSI_Snow_Cover', 'Snow_Albedo_Daily_Tile', 'NDSI']).getInfo()
        bands = np.divide(bands, [100,100,10000] )

        row.extend(bands)

        datalist.append(row)
      
      except TimeoutException:
        print(f"Request Timeout for cell_id {traindf.cell_id[i]}")
      
      except Exception as e:
        print("Some other Error")
      else: 
        signal.alarm(0)
        still_working = False
        x+=1
        if x % 100 == 0:
          print(f'{x} out of {len(traindf.SWE)} complete')

  data = pd.DataFrame(datalist, columns = ['cell_id', 'date', f'{modis}_SnowCover', 
                                           f'{modis}_Albedo', f'{modis}_NDSI'])
  print(data)
  traindf = traindf.merge(data, how = 'left', on=['cell_id', 'date'])
  return traindf

In [69]:
start = time.time()
traindf = pull_MODIS_list(traindf, modis = "MOD10A1")
print(f'Total Time: {time.time() - start}')
print()

start = time.time()
traindf = pull_MODIS_list(traindf, modis = "MYD10A1")
print(f'Total Time: {time.time() - start}')


#!zip -r '/content/drive/MyDrive/snowcapstone team spring 2022/MODIS_Data/MOD10A1_sierras.zip'  '/content/drive/MyDrive/snowcapstone team spring 2022/MODIS_Data/MOD10A1/'
#!zip -r '/content/drive/MyDrive/snowcapstone team spring 2022/MODIS_Data/MYD10A1_sierras.zip'  '/content/drive/MyDrive/snowcapstone team spring 2022/MODIS_Data/MYD10A1/'

100 out of 129805 complete
200 out of 129805 complete
300 out of 129805 complete
400 out of 129805 complete
500 out of 129805 complete
600 out of 129805 complete
700 out of 129805 complete
800 out of 129805 complete
900 out of 129805 complete
1000 out of 129805 complete
1100 out of 129805 complete
1200 out of 129805 complete
1300 out of 129805 complete
1400 out of 129805 complete
1500 out of 129805 complete
1600 out of 129805 complete
1700 out of 129805 complete
1800 out of 129805 complete
1900 out of 129805 complete
2000 out of 129805 complete
2100 out of 129805 complete
2200 out of 129805 complete
2300 out of 129805 complete
2400 out of 129805 complete
2500 out of 129805 complete
2600 out of 129805 complete
2700 out of 129805 complete
2800 out of 129805 complete
2900 out of 129805 complete
Request Timeout for cell_id 74f8c8bc-2f60-4232-b447-43459d5d22f0
3000 out of 129805 complete
3100 out of 129805 complete
3200 out of 129805 complete
3300 out of 129805 complete
3400 out of 129805 c

26800 out of 129805 complete
26900 out of 129805 complete
27000 out of 129805 complete
27100 out of 129805 complete
27200 out of 129805 complete
27300 out of 129805 complete
27400 out of 129805 complete
27500 out of 129805 complete
27600 out of 129805 complete
27700 out of 129805 complete
27800 out of 129805 complete
27900 out of 129805 complete
28000 out of 129805 complete
28100 out of 129805 complete
28200 out of 129805 complete
28300 out of 129805 complete
28400 out of 129805 complete
28500 out of 129805 complete
28600 out of 129805 complete
28700 out of 129805 complete
28800 out of 129805 complete
28900 out of 129805 complete
29000 out of 129805 complete
29100 out of 129805 complete
29200 out of 129805 complete
29300 out of 129805 complete
29400 out of 129805 complete
29500 out of 129805 complete
29600 out of 129805 complete
29700 out of 129805 complete
29800 out of 129805 complete
29900 out of 129805 complete
30000 out of 129805 complete
30100 out of 129805 complete
30200 out of 1

54900 out of 129805 complete
55000 out of 129805 complete
55100 out of 129805 complete
55200 out of 129805 complete
55300 out of 129805 complete
55400 out of 129805 complete
55500 out of 129805 complete
55600 out of 129805 complete
55700 out of 129805 complete
55800 out of 129805 complete
55900 out of 129805 complete
56000 out of 129805 complete
56100 out of 129805 complete
56200 out of 129805 complete
56300 out of 129805 complete
56400 out of 129805 complete
56500 out of 129805 complete
56600 out of 129805 complete
56700 out of 129805 complete
56800 out of 129805 complete
56900 out of 129805 complete
57000 out of 129805 complete
57100 out of 129805 complete
57200 out of 129805 complete
57300 out of 129805 complete
57400 out of 129805 complete
57500 out of 129805 complete
57600 out of 129805 complete
57700 out of 129805 complete
57800 out of 129805 complete
57900 out of 129805 complete
58000 out of 129805 complete
58100 out of 129805 complete
58200 out of 129805 complete
58300 out of 1

82300 out of 129805 complete
82400 out of 129805 complete
82500 out of 129805 complete
82600 out of 129805 complete
82700 out of 129805 complete
82800 out of 129805 complete
82900 out of 129805 complete
83000 out of 129805 complete
83100 out of 129805 complete
83200 out of 129805 complete
83300 out of 129805 complete
83400 out of 129805 complete
83500 out of 129805 complete
83600 out of 129805 complete
83700 out of 129805 complete
83800 out of 129805 complete
83900 out of 129805 complete
84000 out of 129805 complete
84100 out of 129805 complete
84200 out of 129805 complete
84300 out of 129805 complete
84400 out of 129805 complete
84500 out of 129805 complete
84600 out of 129805 complete
84700 out of 129805 complete
84800 out of 129805 complete
84900 out of 129805 complete
85000 out of 129805 complete
85100 out of 129805 complete
85200 out of 129805 complete
85300 out of 129805 complete
85400 out of 129805 complete
85500 out of 129805 complete
85600 out of 129805 complete
85700 out of 1

109400 out of 129805 complete
109500 out of 129805 complete
109600 out of 129805 complete
109700 out of 129805 complete
109800 out of 129805 complete
109900 out of 129805 complete
110000 out of 129805 complete
110100 out of 129805 complete
110200 out of 129805 complete
110300 out of 129805 complete
110400 out of 129805 complete
110500 out of 129805 complete
110600 out of 129805 complete
110700 out of 129805 complete
110800 out of 129805 complete
110900 out of 129805 complete
111000 out of 129805 complete
111100 out of 129805 complete
111200 out of 129805 complete
111300 out of 129805 complete
111400 out of 129805 complete
111500 out of 129805 complete
111600 out of 129805 complete
111700 out of 129805 complete
111800 out of 129805 complete
111900 out of 129805 complete
112000 out of 129805 complete
112100 out of 129805 complete
112200 out of 129805 complete
112300 out of 129805 complete
112400 out of 129805 complete
112500 out of 129805 complete
112600 out of 129805 complete
112700 out

...

16600 out of 129805 complete
16700 out of 129805 complete
16800 out of 129805 complete
16900 out of 129805 complete
17000 out of 129805 complete
17100 out of 129805 complete
17200 out of 129805 complete
17300 out of 129805 complete
17400 out of 129805 complete
17500 out of 129805 complete
17600 out of 129805 complete
17700 out of 129805 complete
17800 out of 129805 complete
17900 out of 129805 complete
18000 out of 129805 complete
18100 out of 129805 complete
18200 out of 129805 complete
18300 out of 129805 complete
18400 out of 129805 complete
18500 out of 129805 complete
18600 out of 129805 complete
18700 out of 129805 complete
18800 out of 129805 complete
Request Timeout for cell_id 7b7a5861-0f59-4efb-b07c-f03b7b85d8c9
18900 out of 129805 complete
19000 out of 129805 complete
19100 out of 129805 complete
19200 out of 129805 complete
19300 out of 129805 complete
19400 out of 129805 complete
19500 out of 129805 complete
19600 out of 129805 complete
19700 out of 129805 complete
19800 o

39900 out of 129805 complete
40000 out of 129805 complete
40100 out of 129805 complete
40200 out of 129805 complete
40300 out of 129805 complete
40400 out of 129805 complete
Request Timeout for cell_id ASO_50M_SWE_USCAKC_157
Request Timeout for cell_id ASO_50M_SWE_USCAKC_244
Request Timeout for cell_id ASO_50M_SWE_USCAKC_251
40500 out of 129805 complete
40600 out of 129805 complete
40700 out of 129805 complete
40800 out of 129805 complete
40900 out of 129805 complete
41000 out of 129805 complete
41100 out of 129805 complete
41200 out of 129805 complete
41300 out of 129805 complete
41400 out of 129805 complete
41500 out of 129805 complete
41600 out of 129805 complete
41700 out of 129805 complete
41800 out of 129805 complete
41900 out of 129805 complete
42000 out of 129805 complete
42100 out of 129805 complete
42200 out of 129805 complete
42300 out of 129805 complete
42400 out of 129805 complete
42500 out of 129805 complete
42600 out of 129805 complete
42700 out of 129805 complete
42800 

66100 out of 129805 complete
66200 out of 129805 complete
66300 out of 129805 complete
66400 out of 129805 complete
66500 out of 129805 complete
66600 out of 129805 complete
66700 out of 129805 complete
66800 out of 129805 complete
66900 out of 129805 complete
67000 out of 129805 complete
67100 out of 129805 complete
67200 out of 129805 complete
67300 out of 129805 complete
67400 out of 129805 complete
67500 out of 129805 complete
67600 out of 129805 complete
67700 out of 129805 complete
67800 out of 129805 complete
67900 out of 129805 complete
68000 out of 129805 complete
68100 out of 129805 complete
68200 out of 129805 complete
68300 out of 129805 complete
68400 out of 129805 complete
68500 out of 129805 complete
68600 out of 129805 complete
68700 out of 129805 complete
68800 out of 129805 complete
68900 out of 129805 complete
69000 out of 129805 complete
69100 out of 129805 complete
69200 out of 129805 complete
69300 out of 129805 complete
69400 out of 129805 complete
69500 out of 1

93100 out of 129805 complete
93200 out of 129805 complete
93300 out of 129805 complete
93400 out of 129805 complete
93500 out of 129805 complete
93600 out of 129805 complete
93700 out of 129805 complete
93800 out of 129805 complete
93900 out of 129805 complete
94000 out of 129805 complete
94100 out of 129805 complete
94200 out of 129805 complete
94300 out of 129805 complete
94400 out of 129805 complete
94500 out of 129805 complete
94600 out of 129805 complete
94700 out of 129805 complete
94800 out of 129805 complete
94900 out of 129805 complete
95000 out of 129805 complete
95100 out of 129805 complete
95200 out of 129805 complete
95300 out of 129805 complete
95400 out of 129805 complete
95500 out of 129805 complete
95600 out of 129805 complete
95700 out of 129805 complete
95800 out of 129805 complete
95900 out of 129805 complete
96000 out of 129805 complete
96100 out of 129805 complete
96200 out of 129805 complete
96300 out of 129805 complete
96400 out of 129805 complete
96500 out of 1

119600 out of 129805 complete
119700 out of 129805 complete
119800 out of 129805 complete
119900 out of 129805 complete
120000 out of 129805 complete
120100 out of 129805 complete
120200 out of 129805 complete
120300 out of 129805 complete
120400 out of 129805 complete
120500 out of 129805 complete
120600 out of 129805 complete
120700 out of 129805 complete
120800 out of 129805 complete
120900 out of 129805 complete
121000 out of 129805 complete
121100 out of 129805 complete
121200 out of 129805 complete
121300 out of 129805 complete
121400 out of 129805 complete
121500 out of 129805 complete
121600 out of 129805 complete
121700 out of 129805 complete
121800 out of 129805 complete
121900 out of 129805 complete
122000 out of 129805 complete
122100 out of 129805 complete
122200 out of 129805 complete
122300 out of 129805 complete
122400 out of 129805 complete
122500 out of 129805 complete
122600 out of 129805 complete
122700 out of 129805 complete
122800 out of 129805 complete
122900 out

In [70]:
pd.DataFrame(traindf).to_csv('~/SnowData/traindf_allregions.csv')

In [71]:
traindf

,cell_id,date,SWE,region,geometry,mean_inversed_swe,mean_local_swe,median_local_swe,max_local_swe,min_local_swe,...,datestring,centroid,center_lat,center_long,MOD10A1_SnowCover,MOD10A1_Albedo,MOD10A1_NDSI,MYD10A1_SnowCover,MYD10A1_Albedo,MYD10A1_NDSI
0,018cf1a1-f945-4097-9c47-0c4690538bb5,2016-01-05,16.400000,sierras,"POLYGON ((-120.61440 39.67242, -120.61440 39.6...",17.255057,10.348000,10.800000,23.060000,0.00,...,2016005,POINT (-13427233.415 4052198.575),39.675880,-120.618890,0.000000,0.000000,0.426766,0.000000,0.000000,0.337222
1,01be2cc7-ef77-4e4d-80ed-c4f8139162c3,2016-01-05,21.100000,sierras,"POLYGON ((-119.60829 38.27575, -119.60829 38.2...",18.306146,11.260000,9.300000,21.600000,4.70,...,2016005,POINT (-13315233.415 3931511.385),38.279274,-119.612777,0.000000,0.000000,0.377328,0.000000,0.000000,0.262573
2,04b7603c-26c6-4004-a8bc-58b2b60e810e,2016-01-05,3.500000,sierras,"POLYGON ((-118.34166 36.56361, -118.34166 36.5...",4.385146,4.712000,3.440000,7.250000,2.52,...,2016005,POINT (-13174233.415 3780427.693),36.567220,-118.346152,0.000000,0.000000,0.299294,0.000000,0.000000,0.547948
3,147d5eb4-e574-47e4-994a-8a2908c06050,2016-01-05,11.700000,sierras,"POLYGON ((-120.87491 39.78297, -120.87491 39.7...",12.326194,11.140000,10.800000,23.060000,3.96,...,2016005,POINT (-13456233.415 4061649.910),39.786417,-120.879401,0.000000,0.000000,0.378600,0.000000,0.000000,0.432753
4,174e3100-c30e-46a4-ac7c-30cd521fc390,2016-01-05,12.900000,sierras,"POLYGON ((-119.54540 37.63117, -119.54540 37.6...",11.383223,11.696000,10.500000,20.380000,7.32,...,2016005,POINT (-13308233.415 3875031.658),37.634731,-119.549895,0.000000,0.000000,0.300411,0.000000,0.000000,0.272249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129800,ASO_50M_SWE_USCALB_79,2018-06-01,0.474991,sierras,"POLYGON ((-118.99236 37.58734, -118.99236 37.5...",2.126230,1.906877,1.888571,3.574384,0.00,...,2018152,POINT (-13245539.186 3870462.847),37.582833,-118.986703,0.050713,0.056434,0.044083,0.018015,0.023185,-0.121343
129801,ASO_50M_SWE_USCALB_87,2018-06-01,1.828016,sierras,"POLYGON ((-118.98152 37.60555, -118.98152 37.5...",2.115175,1.906877,1.888571,3.574384,0.00,...,2018152,POINT (-13244332.105 3872066.046),37.601040,-118.975860,0.028636,0.053653,-0.007782,0.000000,0.000000,-0.069880
129802,ASO_50M_SWE_USCALB_88,2018-06-01,0.114890,sierras,"POLYGON ((-118.98128 37.59654, -118.98128 37.5...",2.125414,1.906877,1.888571,3.574384,0.00,...,2018152,POINT (-13244305.576 3871272.887),37.592032,-118.975621,0.143382,0.132695,0.066765,0.009245,0.012961,-0.126452
129803,ASO_50M_SWE_USCALB_89,2018-06-01,2.170014,sierras,"POLYGON ((-118.98104 37.58753, -118.98104 37.5...",2.135945,1.906877,1.888571,3.574384,0.00,...,2018152,POINT (-13244279.059 3870479.633),37.583024,-118.975383,0.115417,0.143560,0.091611,0.010301,0.015503,-0.018298


# Adding Copernicus Data

In [46]:
traindf = pd.read_csv('~/SnowData/traindf_allregions.csv').drop('Unnamed: 0', axis = 1)
traindf['geometry'] = traindf['geometry'].apply(wkt.loads)

traindf = gpd.GeoDataFrame(traindf, geometry = 'geometry' , crs ="EPSG:4326")

In [53]:
if not os.path.exists('/home/ubuntu/SnowData/CopernicusData'):
    os.makedirs('/home/ubuntu/SnowData/CopernicusData')
else:
    print("Folder Already Exists")

In [54]:
def get_copernicus(traindf, overwrite = False, signal_timer =5):
  start = time.time()
  traindf["copernicus_filelocations"] = "blank"
  
  errored_cells = []

  x = 0
  length_cell_id = len(traindf.cell_id.unique())

  still_working = True
  while still_working:
    try:
      client = Client.open(
      "https://planetarycomputer.microsoft.com/api/stac/v1",
      ignore_conformance=True,
      )
  
    except Exception as e:
      print("Some Error with Image Collection")
    else: 
      still_working = False  

  for i in traindf.cell_id.unique():
    if x % 100 == 1:
      start = time.time()
    error_num = 0
    still_working = True
    while still_working:
      
      signal.alarm(signal_timer)
      try:
        #create a name for the image
        pict_name = i + '_' + 'copernicus'

        #create the whole filename with path to the correct folder
        filename = os.path.join('/home/ubuntu/SnowData/CopernicusData', pict_name)

        # Adapted from https://planetarycomputer.microsoft.com/dataset/cop-dem-glo-90#Example-Notebook :
        
        if not os.path.exists(filename + '.png') or overwrite:
          if error_num == 0:
              area_of_interest = {
              "type": "Polygon",
              "coordinates": [list(traindf.loc[traindf.cell_id == i].iloc[0].geometry.exterior.coords)],
              }
                
              bbox = rasterio.features.bounds(area_of_interest)
              search = client.search(
                collections=["cop-dem-glo-30"],
                bbox = bbox,
                )
                
              min_lon = traindf.loc[traindf.cell_id == i].iloc[0].geometry.bounds[0]
              min_lat = traindf.loc[traindf.cell_id == i].iloc[0].geometry.bounds[1]
              max_lon = traindf.loc[traindf.cell_id == i].iloc[0].geometry.bounds[2]
              max_lat = traindf.loc[traindf.cell_id == i].iloc[0].geometry.bounds[3]
            
          elif error_num > 1:
              area_of_interest = {
              "type": "Polygon",
              "coordinates": [list(shapely.affinity.scale(traindf.loc[traindf.cell_id == i].iloc[0].geometry,
                                                          xfact=(1.+ error_num), 
                                                          yfact=(1.+ error_num)).exterior.coords)],
              }
              print(f"Boundaries scaled by {(error_num*100)} %")
              bbox = rasterio.features.bounds(area_of_interest)
              search = client.search(
                collections=["cop-dem-glo-30"],
                intersects= area_of_interest,
                )
            
              min_lon = shapely.affinity.scale(traindf.loc[traindf.cell_id == i].iloc[0].geometry,
                                                          xfact=(1.+ error_num), 
                                                          yfact=(1.+ error_num)).bounds[0]
              min_lat = shapely.affinity.scale(traindf.loc[traindf.cell_id == i].iloc[0].geometry,
                                                          xfact=(1.+ error_num), 
                                                          yfact=(1.+ error_num)).bounds[1]
              max_lon = shapely.affinity.scale(traindf.loc[traindf.cell_id == i].iloc[0].geometry,
                                                          xfact=(1.+ error_num), 
                                                          yfact=(1.+ error_num)).bounds[2]
              max_lat = shapely.affinity.scale(traindf.loc[traindf.cell_id == i].iloc[0].geometry,
                                                          xfact=(1.+ error_num), 
                                                          yfact=(1.+ error_num)).bounds[3]

          

          
          items = list(search.get_items())

          signed_asset = planetary_computer.sign(items[0].assets["data"])
          
          data = (
              rioxarray.open_rasterio(signed_asset.href)
              .squeeze()
              .drop("band")
          )


          mask_lon = (data.x >= min_lon) & (data.x <= max_lon)
          mask_lat = (data.y >= min_lat) & (data.y <= max_lat)


          cropped_data = data.where(mask_lon & mask_lat, drop=True)

          #hillshade = xrspatial.hillshade(cropped_data)
          img = stack(shade(cropped_data, cmap="red"), 
                      shade(xrspatial.slope(cropped_data), cmap="blue"),
                      shade(xrspatial.aspect(cropped_data), cmap="green"))
          
          export_image(img=img, filename=filename, background=None)
      
      except TimeoutException:
          print(f"Request Timeout for cell_id {i}")
        
      except Exception as e:
        print(f"Some other Error with cell_id {i}")
        print(e)
        error_num += 1
        if error_num > 10:
            errored_cells.extend(i)
            still_working = False
            pass
            
      else: 
        signal.alarm(0)
        still_working = False

        traindf.loc[traindf.cell_id == i, "copernicus_filelocations"] = filename + '.png'
        if x % 100 == 0:
          print(f'{x} out of {length_cell_id} complete')
          print(f'Iteration Time: {time.time() - start}')
        x += 1
  
  print("Zipping")
  shutil.make_archive("new_copernicus", 'zip', '/home/ubuntu/SnowData/CopernicusData')
  print("Zipping Complete")
  print(errored_cells)

In [64]:
get_copernicus(traindf, False)

0 out of 23370 complete
Iteration Time: 0.7547895908355713
100 out of 23370 complete
Iteration Time: 0.7115967273712158
200 out of 23370 complete
Iteration Time: 0.7106237411499023
300 out of 23370 complete
Iteration Time: 0.7117269039154053
400 out of 23370 complete
Iteration Time: 0.7106828689575195
500 out of 23370 complete
Iteration Time: 0.7107019424438477
600 out of 23370 complete
Iteration Time: 0.7109262943267822
700 out of 23370 complete
Iteration Time: 0.7115812301635742
800 out of 23370 complete
Iteration Time: 0.7112069129943848
900 out of 23370 complete
Iteration Time: 0.711418867111206
1000 out of 23370 complete
Iteration Time: 0.7107851505279541
1100 out of 23370 complete
Iteration Time: 0.7110800743103027
1200 out of 23370 complete
Iteration Time: 0.7110652923583984
1300 out of 23370 complete
Iteration Time: 0.7115225791931152
1400 out of 23370 complete
Iteration Time: 0.7109088897705078
1500 out of 23370 complete
Iteration Time: 0.7099449634552002
1600 out of 23370 com

13300 out of 23370 complete
Iteration Time: 0.7476890087127686
13400 out of 23370 complete
Iteration Time: 0.7475113868713379
13500 out of 23370 complete
Iteration Time: 0.7478787899017334
13600 out of 23370 complete
Iteration Time: 0.7481997013092041
13700 out of 23370 complete
Iteration Time: 0.7475457191467285
13800 out of 23370 complete
Iteration Time: 0.7478258609771729
13900 out of 23370 complete
Iteration Time: 0.7478809356689453
14000 out of 23370 complete
Iteration Time: 0.7473902702331543
14100 out of 23370 complete
Iteration Time: 0.7478969097137451
14200 out of 23370 complete
Iteration Time: 0.7472186088562012
14300 out of 23370 complete
Iteration Time: 0.7473187446594238
14400 out of 23370 complete
Iteration Time: 0.7362861633300781
14500 out of 23370 complete
Iteration Time: 0.7058498859405518
14600 out of 23370 complete
Iteration Time: 0.7100014686584473
14700 out of 23370 complete
Iteration Time: 0.7105214595794678
14800 out of 23370 complete
Iteration Time: 0.710735797

In [72]:
traindf = traindf.drop(["region", 'geometry', 'datestring', 'centroid', 'center_lat', 'center_long'], axis = 1)
traindf.head()

,cell_id,date,SWE,mean_inversed_swe,mean_local_swe,median_local_swe,max_local_swe,min_local_swe,mean_local_elevation,median_local_elevation,max_local_elevation,min_local_elevation,copernicus_filelocations,MOD10A1_SnowCover,MOD10A1_Albedo,MOD10A1_NDSI,MYD10A1_SnowCover,MYD10A1_Albedo,MYD10A1_NDSI
0,018cf1a1-f945-4097-9c47-0c4690538bb5,2016-01-05,16.4,17.255057,10.348,10.80,23.06,0.00,1978.761600,2011.680000,2194.56,1609.344,/home/ubuntu/SnowData/CopernicusData/018cf1a1-...,0.0,0.0,0.426766,0.0,0.0,0.337222
1,01be2cc7-ef77-4e4d-80ed-c4f8139162c3,2016-01-05,21.1,18.306146,11.260,9.30,21.60,4.70,2656.027189,2673.095947,2926.08,2194.560,/home/ubuntu/SnowData/CopernicusData/01be2cc7-...,0.0,0.0,0.377328,0.0,0.0,0.262573
2,04b7603c-26c6-4004-a8bc-58b2b60e810e,2016-01-05,3.5,4.385146,4.712,3.44,7.25,2.52,2913.888000,3093.720000,3474.72,2331.720,/home/ubuntu/SnowData/CopernicusData/04b7603c-...,0.0,0.0,0.299294,0.0,0.0,0.547948
3,147d5eb4-e574-47e4-994a-8a2908c06050,2016-01-05,11.7,12.326194,11.140,10.80,23.06,3.96,1961.083200,1975.104000,2225.04,1609.344,/home/ubuntu/SnowData/CopernicusData/147d5eb4-...,0.0,0.0,0.378600,0.0,0.0,0.432753
4,174e3100-c30e-46a4-ac7c-30cd521fc390,2016-01-05,12.9,11.383223,11.696,10.50,20.38,7.32,2447.544000,2407.920000,2987.04,2103.120,/home/ubuntu/SnowData/CopernicusData/174e3100-...,0.0,0.0,0.300411,0.0,0.0,0.272249


In [73]:
pd.DataFrame(traindf).to_csv('~/SnowData/traindf_allregions_final_NoGeom.csv')